# Introduction to Chainer

- この資料はChainer公式チュートリアル（v1.5用）の [Introduction to Chainer](http://docs.chainer.org/en/latest/tutorial/basic.html) をテキトーにまとめて、コードをペタペタ貼ったものです。
- [PyData.Okinawa meetup #9](http://pydataokinawa.connpass.com/event/24769/) での発表の補助に使ったものなので文書よりコードがメインです。
- しっかりと学びたい方、英語が読める方は[公式チュートリアル](http://docs.chainer.org/en/stable/tutorial/index.html)をお読みください。

# リソース

- [Chainer 公式サイト](http://chainer.org/)
- [GitHub – pfnet/chainer](https://github.com/pfnet/chainer)
- [Chainer Documentation](http://docs.chainer.org/en/latest/)
- [Chainer User Group @ Google Groups](https://groups.google.com/forum/#!forum/chainer)
- [Deep Learning のフレームワーク Chainer を公開しました](http://research.preferred.jp/2015/06/deep-learning-chainer/)
- [Chainerチュートリアル -v1.5向け- ViEW2015](http://www.slideshare.net/ryokuta/chainer-v15-view2015)

## Chainerの思想

- 柔軟性
- 他のフレームワーク（Caffe, Theano, Torch）
    - **Define-and-Run**方式
    - 最初にカチッとネットワークを決めてから走らせる。
- Chainer
    - **Define-by-Run**方式
    - ネットワークは順計算のときに動的に定義される。
    - プログラミングのロジックを保存するのではなく、計算履歴を保存する
- Define-by-Run方式は複数GPUの並列化を簡単にする。

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

順計算を始めるには、入力配列をVariableオブジェクトにセットする必要がある。

In [2]:
x_data = np.array([5], dtype=np.float32)

In [3]:
x = Variable(x_data)

注：現時点（2015/07）でのChainerは32ビットのfloatしかサポートしていない。

In [4]:
y  = x ** 2 - 2 * x + 1

In [5]:
y

In [6]:
y.data

array([ 16.], dtype=float32)

yが持っているのは結果としての値だけではなく、「計算の履歴」or computational graphを持っている。これにより、backward()で微分を計算できるようになる。

In [7]:
y.backward()

This runs error backpropagation (a.k.a. backprop or reverse-mode automatic differentiation

In [8]:
x.grad

array([ 8.], dtype=float32)

In [9]:
y.data

array([ 16.], dtype=float32)

Chainerはデフォルトで処理の途中で現れる変数の勾配を捨てる（メモリ効率のため）。この勾配情報を保持するには`retain_grad＝True`を`backward`メソッドに渡せば良い。


In [10]:
z = 2*x

In [11]:
y = x**2 - z + 1

In [12]:
# 途中の変数の勾配情報を保持
y.backward(retain_grad=True)

In [13]:
z.grad

array([-1.], dtype=float32)

In [14]:
x.grad

array([ 16.], dtype=float32)

多次元配列でやってみよう

In [15]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32))
#x = Variable(np.array([5], dtype=np.float32))

In [16]:
y = x ** 2 - 2 * x + 1

In [17]:
# Set initial error
y.grad = np.ones((2, 3), dtype=np.float32)

In [18]:
y.backward()

In [19]:
x.grad

array([[  0.,   2.,   4.],
       [  6.,   8.,  10.]], dtype=float32)

# リンク（Links）

- 一般的な流れ
    - パラメータ付き関数を定義
    - パラメータを最適化
- リンクはパラメータ（最適化のターゲット）を持つオブジェクト
- 一番基本的なリンクはパラメータ付き関数
- リンクは chainer v1.4まで使われていたparameterized functionsのようなもの
- `Linear`リンク
    - よく使われる
    - a.k.a. 全結合層、アファイン変換
    - $f(x) = Wx + b$
    - `linear()`関数は$x, W, b$を受け付ける純粋関数で、`Linear`リンクに対応
- ほとんどの関数やリンクはミニバッチ形式で入力を受け付ける。この場合、最初の次元がミニバッチのサイズになるので注意すること。

In [20]:
# リニアリンクの作り方
f = F.Linear(3, 2)

In [21]:
f

In [22]:
f.W.data

array([[ 0.50770068,  0.41443419,  0.23775133],
       [-1.36387312, -0.21992619, -0.51988107]], dtype=float32)

In [23]:
f.b.data

array([ 0.,  0.], dtype=float32)

パラメータは`Variable`のインスタンス

In [24]:
f.W

<variable W>

In [25]:
f.b

<variable b>

`Linear`リンクのインスタンスは普通の関数のように振る舞う。

In [26]:
# 変数xを定義
x = Variable(np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32))

In [27]:
y = f(x)

In [28]:
y.data

array([[ 2.04982305, -3.36336875],
       [ 5.52948189, -9.67441082]], dtype=float32)

In [29]:
# 実際にやってる計算（x W' + b）
np.dot(x.data, f.W.data.T) + f.b.data

array([[ 2.04982305, -3.36336875],
       [ 5.52948189, -9.67441082]], dtype=float32)

In [30]:
# 純粋関数 chainer.functions.linearを使用しても同じ値が帰ってくる
F.linear(x, f.W, f.b).data

array([[ 2.04982305, -3.36336875],
       [ 5.52948189, -9.67441082]], dtype=float32)

- パラメータに対する勾配は`backward()`で計算
- 勾配は上書きされず溜まっていく
- なので最初は勾配を0に初期化する必要あり（zerograds()メソッド）

初期化前

In [31]:
f.W.grad

array([[ nan,  nan,  nan],
       [ nan,  nan,  nan]], dtype=float32)

In [32]:
f.b.grad

array([ nan,  nan], dtype=float32)

初期化

In [33]:
f.zerograds()

初期化後

In [34]:
f.W.grad

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]], dtype=float32)

In [35]:
f.b.grad

array([ 0.,  0.], dtype=float32)

パラメータ勾配を求めるには、この状態でbackward()メソッドを呼べば良い。

In [36]:
# y.gradの初期値をチェック
y.grad

In [37]:
# y.gradの値をdy/dy = 1に初期化
y.grad = np.ones((2, 2), dtype=np.float32)

In [38]:
y.grad

array([[ 1.,  1.],
       [ 1.,  1.]], dtype=float32)

In [39]:
y.backward()

In [40]:
f.W.grad

array([[ 5.,  7.,  9.],
       [ 5.,  7.,  9.]], dtype=float32)

In [41]:
f.b.grad

array([ 2.,  2.], dtype=float32)

In [42]:
x.data

array([[ 1.,  2.,  3.],
       [ 4.,  5.,  6.]], dtype=float32)

返ってきた値の計算方法

$$
y_{i} = \sum_{j=1}^{3} W_{i,j} x_{j} + b_{i}, \quad i \in [1, 2]
$$
$$
\begin{cases}
\frac{y_{i}}{w_{i,j}} &= x_{j}
\\
\frac{y_{i}}{b_{i}} &= 1
\end{cases}
$$
$$
\begin{cases}
\frac{y_{n, i}}{w_{i,j}} = \sum_{n=1}^{N} x_{n, j}
\\
\frac{y_{n, i}}{b_{i}} = \sum_{n=1}^{N} 1 = N
\end{cases}
$$

## chainとしてモデルを記述



In [43]:
l1 = L.Linear(4, 3)

In [44]:
l2 = L.Linear(3, 2)

In [45]:
def my_forward(x):
    h = l1(x)
    return l2(h)

In [46]:
x = Variable(np.arange(8, dtype=np.float32).reshape(2, 4))
x.data

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.]], dtype=float32)

In [47]:
y = my_forward(x)

In [48]:
y.data

array([[-0.54781914, -2.85455251],
       [-1.87642241, -4.82808256]], dtype=float32)

In [49]:
l1.W.data

array([[ 0.16298151, -0.03102867, -0.02969207, -0.15049267],
       [ 0.49399233, -0.53489888, -0.35294393, -0.4406653 ],
       [ 0.41361329, -0.0712271 , -0.14136939, -0.56449163]], dtype=float32)

In [50]:
l2.W.data

array([[-0.31882811,  0.57858658, -0.37227085],
       [ 0.53694582,  0.032618  ,  1.21126509]], dtype=float32)

リンクと処理をクラスにまとめると再利用できて良い

In [51]:
class MyProc(object):
    def __init__(self):
        self.l1 = L.Linear(4, 3)
        self.l2 = L.Linear(3, 2)
        
    def forward(self, x):
        h = self.l1(x)
        return self.l2(h)

`Chain`クラスのサブクラスにするとさらに再利用がしやすくなる。

- パラメータ管理
- CPU/GPUのサポート
- ロバストで柔軟なsave/loadなどなど

In [52]:
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__(
            l1 = L.Linear(4, 3),
            l2 = L.Linear(3, 2),
        )
        
    def __call__(self, x):
        h = self.l1(x)
        y = self.l2(h)
        return y

- l1 や l2 は MyChain の子リンク(child links)と呼ばれる
- Chain自身もLinkを継承している
- MyChainオブジェクトを子リンクとして持つようなことも可能

`ChainList`クラスを使ってチェーンを作ることもできる。これはリンクのリストのようなもの。
- リンク数が可変の時に使うと便利
- リンクの数が決まっている時には、Chainクラスを使った方が良い。

In [53]:
class MyChain2(ChainList):
    def __init__(self):
        super(MyChain2, self).__init__(
            L.Linear(4, 3),
            L.Linear(3, 2),
        )
        
    def __call__(self, x):
        h = self[0](x)
        y = self[1](h)
        return y

## Optimizer

In [54]:
N = 2
x = Variable(np.arange(8).reshape(N, 4).astype(np.float32))
x.data

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.]], dtype=float32)

In [55]:
model = MyChain()

In [56]:
optimizer = optimizers.SGD()

In [57]:
optimizer.setup(model)

### 方法１：手で勾配を計算して、アップデート update()
- update()は引数なし
- 勾配の初期化を忘れずに

In [58]:
model.zerograds()

In [59]:
y = model(x)

In [60]:
# dy/dy = 1
y.grad = np.ones([N, 2], dtype=np.float32)

In [61]:
y.backward()

In [62]:
model.l1.W.data

array([[-0.1211196 , -0.38596928, -0.97663116, -0.46448746],
       [ 0.24993719, -0.70042211,  0.15281869, -0.24290739],
       [-0.37427124, -0.18077426, -0.88798028,  0.21463995]], dtype=float32)

In [63]:
# 勾配計算
optimizer.update()

In [64]:
model.l1.W.data

array([[-0.06843638, -0.30694446, -0.87126476, -0.33277947],
       [ 0.24168897, -0.71279442,  0.13632225, -0.26352796],
       [-0.38020563, -0.18967585, -0.89984906,  0.19980396]], dtype=float32)

### 方法２：update()に損失関数を渡す

- この場合、zerogradsは自動的に呼ばれる。

```
def lossfun(args...):
    ...
    return loss
```

```
optimizer.update(lossfun, args...)
```

パラメータや勾配の操作

- 荷重減衰（weight decay）
- 勾配クリッピング（gradient clipping）

などをつかうには、optimizerにhook functionを追加すればよい。

hook functionは自分で書くこともできる。

In [65]:
# 荷重減衰
optimizer.add_hook(chainer.optimizer.WeightDecay(0.0005))

## Serializer

- `Serializer`はオブジェクトを直列化するためのインターフェース。
- LinkとOptimizerは`Serializer`による直列化をサポート
- serializers moduleの中で定義されている？
- 現在のところ、HDF5形式のserializerとdeserializerがある。
- `serializers.save_hdf5()`
    - parameter と persistent valuesだけ直列化される。
    - 保存したい量に対しては、事前にLink.add_persistent()を呼ぶ。
    - 登録された値は、add_persistentを読んだ時に登録した名前で呼び出せる.
- `serializers.load_hdf5()`

In [66]:
# modelのパラメータを'my.model'というファイルの中にHDF5 formatで保存。
serializers.save_hdf5('my.model', model)

In [67]:
#!open .

In [68]:
serializers.load_hdf5('my.model', model)

optimizerの状態も同じ関数で保存・呼出できる

- 保存されるのは内部状態
- イテレーション回数
- MomentumSGDなどのモメンタム（慣性項）
- ターゲットリンクのパラメータやPersistent valueは保存しない
- 保存状態から学習を再開するには、optmizerとともにtarget linkも保存する必要あり。

In [69]:
serializers.save_hdf5('my.state', optimizer)

In [70]:
#!open .

In [71]:
serializers.load_hdf5('my.state', optimizer)